In [1]:
### 設定一下 jupyter
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dps.settings")
import django

from svapps.dps.models import Revenue, Withdraw
from svapps.operate.nsf_algorithms import (
    get_quote_currency,
    get_withdraw_fixed_rate_dict
)
from svapps.dps.enums import ReportType

In [2]:
withdraw_id = 9535
withdraw = Withdraw.objects.get(pk=withdraw_id)
# 過濾出需要的 revenues
revenues = Revenue.objects.filter(
        withdraw=withdraw,
        report__type=ReportType.RECORDING,
    )

### `read_frame`

df 就是 dataframe 的意思 <br>
以下 `df.head()` 就是印出 dataframe 頭 5 個 row <br>
`read_frame` 用來把 django 的 `queryset` 換成 pandas 的 `dataframe`

In [3]:
from django_pandas.io import read_frame
df = read_frame(revenues, fieldnames=[
        'presented_at', 'track__name', 'track__isrc', 'track__album__name', 'currency__code', 'artist__name',
        'download_quantity', 'streaming_quantity', 'carrier_quantity',
        'download_amount', 'streaming_amount', 'carrier_amount',
        'user_amount', 'our_amount', 'retailer'])

In [4]:
from IPython.display import display, HTML   # 用來在 jupyter 把 dataframe 印出表格, 比較漂亮
HTML(df.head().to_html())

,presented_at,track__name,track__isrc,track__album__name,currency__code,artist__name,download_quantity,streaming_quantity,carrier_quantity,download_amount,streaming_amount,carrier_amount,user_amount,our_amount,retailer
0,2016-12-01,Suga Boom Boom,QZS1Z1632740,Suga Boom Boom,TWD,Down3r,0,0,1,0E-8,0E-8,7.39222222,4.25792000,1.06448000,電信加值：遠傳
1,2016-12-01,Sexy,QZS1Z1631810,Sexy,TWD,Alex Galac,0,0,2,0E-8,0E-8,14.78444444,8.51584000,2.12896000,電信加值：遠傳
2,2016-11-01,Live On (feat. Akello Light),QZS1Z1631666,Live On (feat. Akello Light) [Inc. Remixes],TWD,Groove Govnor & Kurtx & Akello Light,0,0,1,0E-8,0E-8,4.77560976,2.75075122,0.68768781,電信加值：遠傳
3,2016-11-01,Cello Suite 2 Gigue,USLZJ1558154,"Classic Bach, Vol. 1",TWD,8 bit bach,0,0,1,0E-8,0E-8,4.77560976,2.75075122,0.68768781,電信加值：遠傳
4,2016-11-01,Monody (feat. Laura Brehm),QZS1Z1624436,Stay Young,TWD,TheFatRat & Laura Brehm,0,0,1,0E-8,0E-8,4.77560976,2.75075122,0.68768781,電信加值：遠傳


### df.empty <br>
檢查 df 是否爲空

In [5]:
print(df.empty)

False


### astype() <br>
把原本 Decimal 換成 float 方便計算 <br>
http://pandas.pydata.org/pandas-docs/version/0.19.1/generated/pandas.DataFrame.astype.html

In [6]:
df['user_amount'] = df['user_amount'].astype(float)
df['our_amount'] = df['our_amount'].astype(float)

### rename <br>
把原來讀取 queryset 的預設 index 名稱, `currency__code` <br>
換成 `base_currency_code` <br>
http://pandas.pydata.org/pandas-docs/version/0.19.1/generated/pandas.Index.rename.html?highlight=rename#pandas.Index.rename

In [7]:
df.rename(columns={'currency__code': 'base_currency_code'}, inplace=True)
HTML(df.head().to_html())

,presented_at,track__name,track__isrc,track__album__name,base_currency_code,artist__name,download_quantity,streaming_quantity,carrier_quantity,download_amount,streaming_amount,carrier_amount,user_amount,our_amount,retailer
0,2016-12-01,Suga Boom Boom,QZS1Z1632740,Suga Boom Boom,TWD,Down3r,0,0,1,0E-8,0E-8,7.39222222,4.257920,1.064480,電信加值：遠傳
1,2016-12-01,Sexy,QZS1Z1631810,Sexy,TWD,Alex Galac,0,0,2,0E-8,0E-8,14.78444444,8.515840,2.128960,電信加值：遠傳
2,2016-11-01,Live On (feat. Akello Light),QZS1Z1631666,Live On (feat. Akello Light) [Inc. Remixes],TWD,Groove Govnor & Kurtx & Akello Light,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳
3,2016-11-01,Cello Suite 2 Gigue,USLZJ1558154,"Classic Bach, Vol. 1",TWD,8 bit bach,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳
4,2016-11-01,Monody (feat. Laura Brehm),QZS1Z1624436,Stay Young,TWD,TheFatRat & Laura Brehm,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳


### df.apply
Applies function 到每個 row <br>
- 把原本日期格式, 轉換成需要的字串 <br> 
- 拿到對應的匯率 <br><br>
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html

In [8]:
# 把原本日期格式, 轉換成需要的字串 
def df_format_presented_at(row):
    return '{0:d}/{1:d}'.format(row['presented_at'].year, row['presented_at'].month)

# 把 function apply 到每個 row
df['presented_at'] = df.apply(df_format_presented_at, axis=1)
HTML(df.head().to_html())

,presented_at,track__name,track__isrc,track__album__name,base_currency_code,artist__name,download_quantity,streaming_quantity,carrier_quantity,download_amount,streaming_amount,carrier_amount,user_amount,our_amount,retailer
0,2016/12,Suga Boom Boom,QZS1Z1632740,Suga Boom Boom,TWD,Down3r,0,0,1,0E-8,0E-8,7.39222222,4.257920,1.064480,電信加值：遠傳
1,2016/12,Sexy,QZS1Z1631810,Sexy,TWD,Alex Galac,0,0,2,0E-8,0E-8,14.78444444,8.515840,2.128960,電信加值：遠傳
2,2016/11,Live On (feat. Akello Light),QZS1Z1631666,Live On (feat. Akello Light) [Inc. Remixes],TWD,Groove Govnor & Kurtx & Akello Light,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳
3,2016/11,Cello Suite 2 Gigue,USLZJ1558154,"Classic Bach, Vol. 1",TWD,8 bit bach,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳
4,2016/11,Monody (feat. Laura Brehm),QZS1Z1624436,Stay Young,TWD,TheFatRat & Laura Brehm,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳


In [9]:
# 拿到對應的匯率  
# 拿到結算的匯率 dictionary
fixed_rate_dict = get_withdraw_fixed_rate_dict(withdraw.presented_at)

# 拿到這個 withdraw 的 quote currency
quote_currency = get_quote_currency(withdraw.user.profile.payment_company)
# 新建立一個 column 是 quote_currency code 並填入 quote currency code
df['quote_currency_code'] = quote_currency.code


from pprint import pprint
pprint(fixed_rate_dict)

{'CNY': {'CNY': Decimal('1.00000000'),
         'TWD': Decimal('4.38834756'),
         'USD': Decimal('0.14571966')},
 'EUR': {'CNY': Decimal('7.69776061'),
         'TWD': Decimal('33.78044896'),
         'USD': Decimal('1.12171506')},
 'HKD': {'CNY': Decimal('0.88042805'),
         'TWD': Decimal('3.86362426'),
         'USD': Decimal('0.12829568')},
 'JPY': {'CNY': Decimal('0.06185518'),
         'TWD': Decimal('0.27144203'),
         'USD': Decimal('0.00901352')},
 'MYR': {'CNY': Decimal('1.60766809'),
         'TWD': Decimal('7.05500633'),
         'USD': Decimal('0.23426885')},
 'SGD': {'CNY': Decimal('4.96199738'),
         'TWD': Decimal('21.77496909'),
         'USD': Decimal('0.72306057')},
 'THB': {'CNY': Decimal('0.20151794'),
         'TWD': Decimal('0.88433077'),
         'USD': Decimal('0.02936513')},
 'TWD': {'CNY': Decimal('0.22787621'),
         'TWD': Decimal('1.00000000'),
         'USD': Decimal('0.03320604')},
 'USD': {'CNY': Decimal('6.86249200'),
         'TWD':

In [10]:
# 取得對應 row 需要的匯率
# 用每個 row 的 base currency code 跟 quote currency code 
# 在 fixed_rate_dict 裏面找到對應的匯率

def df_get_rate(row, fixed_rate_dict):
    return fixed_rate_dict[row['base_currency_code']][row['quote_currency_code']]

# 新建一個 column rate, 使用 apply 填入每個 row
df['rate'] = df.apply(df_get_rate, args=(fixed_rate_dict,), axis=1)
HTML(df.head().to_html())

,presented_at,track__name,track__isrc,track__album__name,base_currency_code,artist__name,download_quantity,streaming_quantity,carrier_quantity,download_amount,streaming_amount,carrier_amount,user_amount,our_amount,retailer,quote_currency_code,rate
0,2016/12,Suga Boom Boom,QZS1Z1632740,Suga Boom Boom,TWD,Down3r,0,0,1,0E-8,0E-8,7.39222222,4.257920,1.064480,電信加值：遠傳,USD,0.03320604
1,2016/12,Sexy,QZS1Z1631810,Sexy,TWD,Alex Galac,0,0,2,0E-8,0E-8,14.78444444,8.515840,2.128960,電信加值：遠傳,USD,0.03320604
2,2016/11,Live On (feat. Akello Light),QZS1Z1631666,Live On (feat. Akello Light) [Inc. Remixes],TWD,Groove Govnor & Kurtx & Akello Light,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳,USD,0.03320604
3,2016/11,Cello Suite 2 Gigue,USLZJ1558154,"Classic Bach, Vol. 1",TWD,8 bit bach,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳,USD,0.03320604
4,2016/11,Monody (feat. Laura Brehm),QZS1Z1624436,Stay Young,TWD,TheFatRat & Laura Brehm,0,0,1,0E-8,0E-8,4.77560976,2.750751,0.687688,電信加值：遠傳,USD,0.03320604
